In [39]:
import functions200m as fun200
import functions100m as fun100
import functions50m as fun50
import pandas as pd
import tabula

In [43]:
pdf = "pdfs/trofeo%20portuense%202023/ResultList_41.pdf"
tabu = tabula.read_pdf(pdf, pages='all')
tabu = tabu[0]
print(tabu)

                             Prueba 41 Masc., 200m Espalda  \
0                   27/05/2023 - 19:20                 NaN   
1                        Clasificación                  AN   
2              1. MONGE FERRUZ, Marcos                  06   
3               50m: 31.34 31.34 100m:             1:05.98   
4            2. BELINCHON MATOS, Angel                  06   
5               50m: 33.40 33.40 100m:             1:07.48   
6                 3. RUIZ MANGA, Oscar                  08   
7               50m: 33.53 33.53 100m:             1:09.37   
8          4. MERINO RODRIGUEZ, Yeison                  08   
9               50m: 32.79 32.79 100m:             1:10.02   
10        5. GONZALEZ CEBALLOS, Rafael                  08   
11              50m: 33.18 33.18 100m:             1:09.18   
12          6. RODRIGUEZ RAMOS, Alvaro                  09   
13              50m: 32.62 32.62 100m:             1:09.87   
14            7. SANCHEZ ABADÍA, Mario                  09   
15      

In [44]:
df = fun200.pdf_to_df(pdf)


06
6
06
6
08
8
08
8
08
8
09
9
09
9
06
6
09
9
06
6
09
9
02
2
10
0
10
0
09
9
10
0
11

12
2
10
0
   first_surname second_surname       name                       team  \
0          monge         ferruz     marcos                  C.N.Cadiz   
1      belinchon          matos      angel            C.N.Los Barrios   
2           ruiz          manga      oscar             C.D.N.Sanlucar   
3         merino      rodriguez     yeison              C.N.Portuense   
4       gonzalez       ceballos     rafael           C.N.San Fernando   
5      rodriguez          ramos     alvaro           C.N.San Fernando   
6        sanchez         abadía      mario           C.N.Dos Hermanas   
7       pastoril         ibañez      fabio         C.N.Bahia De Cadiz   
8        jimenez          romay     victor              C.N.Portuense   
9       martinez        jimenez     daniel                  C.N.Cadiz   
10      alabarce       manrique     ismael            C.N.Los Barrios   
11        romero           rio

/Users/dankd/Documents/automatization-swimming-data/functions200m.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop("full_name", axis=1, inplace=True)
/Users/dankd/Documents/automatization-swimming-data/functions200m.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file = open("clubs.csv", "r")
/Users/dankd/Documents/automatization-swimming-data/functions200m.py:274: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # remo

In [37]:
pdf = "pdfs/trofeo%20portuense%202023/ResultList_41.pdf"
tabu = tabula.read_pdf(pdf, pages='all')
tabu = tabu[0]
print(tabu)
print(tabu.iloc[:,-1].tolist())

                             Prueba 41 Masc., 200m Espalda  \
0                   27/05/2023 - 19:20                 NaN   
1                        Clasificación                  AN   
2              1. MONGE FERRUZ, Marcos                  06   
3               50m: 31.34 31.34 100m:             1:05.98   
4            2. BELINCHON MATOS, Angel                  06   
5               50m: 33.40 33.40 100m:             1:07.48   
6                 3. RUIZ MANGA, Oscar                  08   
7               50m: 33.53 33.53 100m:             1:09.37   
8          4. MERINO RODRIGUEZ, Yeison                  08   
9               50m: 32.79 32.79 100m:             1:10.02   
10        5. GONZALEZ CEBALLOS, Rafael                  08   
11              50m: 33.18 33.18 100m:             1:09.18   
12          6. RODRIGUEZ RAMOS, Alvaro                  09   
13              50m: 32.62 32.62 100m:             1:09.87   
14            7. SANCHEZ ABADÍA, Mario                  09   
15      

In [38]:
df = fun200.pdf_to_df(pdf)
print(df)

06
6
06
6
08
8
08
8
08
8
09
9
09
9
06
6
09
9
06
6
09
9
02
2
10
0
10
0
09
9
10
0
11

12
2
10
0
   first_surname second_surname       name                       team  \
0          monge         ferruz     marcos                  C.N.Cadiz   
1      belinchon          matos      angel            C.N.Los Barrios   
2           ruiz          manga      oscar             C.D.N.Sanlucar   
3         merino      rodriguez     yeison              C.N.Portuense   
4       gonzalez       ceballos     rafael           C.N.San Fernando   
5      rodriguez          ramos     alvaro           C.N.San Fernando   
6        sanchez         abadía      mario           C.N.Dos Hermanas   
7       pastoril         ibañez      fabio         C.N.Bahia De Cadiz   
8        jimenez          romay     victor              C.N.Portuense   
9       martinez        jimenez     daniel                  C.N.Cadiz   
10      alabarce       manrique     ismael            C.N.Los Barrios   
11        romero           rio

/Users/dankd/Documents/automatization-swimming-data/functions200m.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop("full_name", axis=1, inplace=True)
/Users/dankd/Documents/automatization-swimming-data/functions200m.py:255: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file = open("clubs.csv", "r")
/Users/dankd/Documents/automatization-swimming-data/functions200m.py:274: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # remo